In [ ]:
from fastai.vision.all import *
import numpy as np
from ipywidgets import widgets
from pandas.api.types import CategoricalDtype
import pandas as pd
from ipywidgets import widgets
from pandas.api.types import CategoricalDtype
import os
import matplotlib as mpl

In [ ]:
df=pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
print(len(df))
print(df.columns)
print(df['label'].value_counts().plot.bar())

In [ ]:
df.head()

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
Path.BASE_PATH=path
path.ls()



In [ ]:
get_items=get_image_files(path/'train_images')

In [ ]:
Image.open(get_items[1])

In [ ]:

#Check the images file, if there are a few that are corrupt
failed = verify_images(get_items)
failed

## show the Categoories 

In [ ]:
count_dict = df.label.value_counts()
count_dict

In [ ]:
df.info

In [ ]:
def get_x(r): return path/'train_images'/r['image_id']
def get_y(r): return r['label']



In [ ]:
get_y(df)

In [ ]:
dblock = DataBlock(blocks=(ImageBlock,CategoryBlock),
                get_x=get_x, 
                get_y=get_y,
                batch_tfms=[*aug_transforms(min_scale=0.5, size=128),
                Normalize.from_stats(*imagenet_stats)],
                item_tfms = RandomResizedCrop(224, min_scale=0.35))
                   


In [ ]:
dls = dblock.dataloaders(df,bs=128)
dls.show_batch()

In [ ]:
list(dls)

In [ ]:
dls.valid.show_batch()

In [ ]:

#Dataset
dsets = dblock.datasets(df)
dsets.train[0]
x,y = dsets.train[0]
x,y

In [ ]:
dblock.summary(df)


In [ ]:
dl_train=dls.train
dl_valid=dls.valid
len(dl_train),len(dl_valid)

In [ ]:
xb,yb = first(dls[0])
xb.shape,yb.shape
#Batch_size contains 128 images
#Images Pixels are 128*128
#3 channel

## Check the torch size

In [ ]:
xb,yb = first(dls.valid)
print("Print the Input and Output Shape: \n", xb.shape,yb.shape)

In [ ]:
#Categories number 
#output Categories
dls.c

#Input has a 128 Pixel

In [ ]:
# I try to grabb a mini batch from our DataLoader and then passing it to the model:
#we have a batch side of 64 and 5 categories 
x,y = to_cpu(dls.train.one_batch())
subset_model = learn.model(x)
print("shape of the Submodel: \n"subset_model.shape
subset_model[0]

In [ ]:
learn = cnn_learner(dls, resnet18,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fine_tune(4, base_lr=1e-3, freeze_epochs=4)

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4, base_lr=1e-3, freeze_epochs=4)

In [ ]:
#79%
learn = cnn_learner(dls, resnet50,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fine_tune(5, base_lr=1e-3, freeze_epochs=4)

In [ ]:
learn = cnn_learner(dls, resnet101,
                    loss_func=F.cross_entropy, metrics=accuracy)
learn.fine_tune(4, base_lr=1e-3, freeze_epochs=4)

In [ ]:
learn.lr_find()

#  I defined a new Model

In [ ]:
def conv(ni, nf, ks=3, act=True):
    res = nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

In [ ]:
simple_cnn = sequential(
    conv(3 ,4),            #14x14
    conv(4 ,8),            #7x7
    conv(8 ,16),           #4x4
    conv(16,32),           #2x2
    conv(32,5, act=False), #1x1
    Flatten(),
)

In [ ]:
learn = Learner(dls, simple_cnn, loss_func=F.cross_entropy, metrics=accuracy)

In [ ]:
learn.summary()

In [ ]:
learn.fit_one_cycle(2, 0.01)


In [ ]:
m = learn.model[0]
m

In [ ]:
m[0].weight.shape

In [ ]:
m[0].bias.shape

In [ ]:
def fit(epochs=1):
    learn = Learner(dls, simple_cnn, loss_func=F.cross_entropy,
                    metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit(epochs, 0.06)
    return learn

In [ ]:
learn = fit()

In [ ]:
learn.activation_stats.plot_layer_stats(0)